In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter

from src.io.datasets import CSVReader
from src.config.settings import get_settings

%matplotlib widget

In [2]:
settings = get_settings()

reader = CSVReader()

In [3]:
disguise_results = reader.read(settings.WEIGHTED_PLAY_DISGUISE_RESULTS_FILE)

In [4]:
# Only players with more than 50 plays
play_counts = disguise_results.groupby('nflId').size()
players_with_enough_plays = play_counts[play_counts > 50].index
filtered_data = disguise_results[disguise_results['nflId'].isin(players_with_enough_plays)]

In [5]:
mean_disguise = filtered_data.groupby(['nflId'])['disguise_score'].mean()
filtered_data = filtered_data.merge(mean_disguise, on='nflId', suffixes=('', '_mean'))
filtered_data = filtered_data.drop_duplicates(subset=['nflId'], keep='first')
filtered_data = filtered_data[['jerseyNumber', 'club', 'position', 'displayName', 'disguise_score_mean']]

In [6]:
filtered_data = filtered_data.sort_values('disguise_score_mean', ascending=False)
filtered_data = filtered_data.rename(columns={
    'position': 'Position',
    'jerseyNumber': 'Number',
    'displayName': 'Name',
    'club': 'Team',
    'disguise_score_mean': 'Disguise Score',
})

In [7]:
top_10 = filtered_data.head(10)
bottom_10 = filtered_data.tail(10)

In [8]:
top_10.reset_index(drop=True)

,Number,Team,Position,Name,Disguise Score
0,3,NE,SS,Jabrill Peppers,0.203088
1,24,NYG,SS,Dane Belton,0.197148
2,27,DAL,FS,Jayron Kearse,0.197089
3,23,NE,FS,Kyle Dugger,0.190051
4,30,TB,CB,Dee Delaney,0.175012
5,37,TEN,FS,Amani Hooker,0.171245
6,24,LV,SS,Johnathan Abram,0.150227
7,38,KC,CB,L'Jarius Sneed,0.134355
8,1,SF,FS,Jimmie Ward,0.131625
9,38,CAR,FS,Myles Hartsfield,0.118658


In [9]:
bottom_10.reset_index(drop=True)

,Number,Team,Position,Name,Disguise Score
0,24,HOU,CB,Derek Stingley,0.001605
1,24,LA,SS,Taylor Rapp,0.001495
2,29,CIN,CB,Cam Taylor-Britt,0.001405
3,24,MIN,FS,Camryn Bynum,0.001331
4,2,PHI,CB,Darius Slay,0.001020
5,24,PHI,CB,James Bradberry,0.000264
6,7,MIN,CB,Patrick Peterson,0.000114
7,6,LA,CB,Derion Kendrick,0.000059
8,26,CLE,CB,Greedy Williams,0.000058
9,25,IND,FS,Rodney Thomas,0.000019


In [10]:
from plottable import ColumnDefinition, Table
from plottable.plots import image
from plottable.cmap import normed_cmap
from matplotlib.colors import LinearSegmentedColormap

df = top_10.copy()

df = df[['Name', 'Position', 'Team', 'Disguise Score']]

team_abbreviations = df['Team'].unique()

team_logos = {team: f"../assets/logos/{team}.png" for team in team_abbreviations}

df['Team'] = df['Team'].map(team_logos)
df = df.set_index("Name")
df['Disguise Score'] = df['Disguise Score'].round(4)

cmap = LinearSegmentedColormap.from_list(
    name="bugw", colors=["#ffffff", "#f2fbd2", "#c9ecb4", "#93d3ab", "#35b0ab"], N=256
)

name_column = ColumnDefinition("Name", textprops={"ha": "left"}, width=0.3, group="Player")
position_column = ColumnDefinition("Position", textprops={"ha": "center"}, width=0.15, group="Player")
team_column = ColumnDefinition("Team", plot_fn=image, width=0.15, textprops={"ha": "center"}, group="Player")
disguise_column = ColumnDefinition("Disguise Score",
            cmap=normed_cmap(df["Disguise Score"], cmap=cmap), width=0.4)

fig, ax = plt.subplots()
plt.tight_layout(pad=0.5)
table = Table(df, column_definitions=[name_column, position_column, team_column, disguise_column])
fig.savefig("../reports/figures/table_1_left.png", facecolor=ax.get_facecolor(), dpi=200, bbox_inches='tight')
plt.close(fig)

In [11]:
df = bottom_10.copy()

df = df[['Name', 'Position', 'Team', 'Disguise Score']]

team_abbreviations = df['Team'].unique()

team_logos = {team: f"../assets/logos/{team}.png" for team in team_abbreviations}

df['Team'] = df['Team'].map(team_logos)
df = df.set_index("Name")
df['Disguise Score'] = df['Disguise Score'].round(4)

cmap_red_to_white = LinearSegmentedColormap.from_list(
    name="red_to_white", 
    colors=["#ff0000", "#ffffff"],  # Red to white
    N=256
)

name_column = ColumnDefinition("Name", textprops={"ha": "left"}, width=0.3, group="Player")
position_column = ColumnDefinition("Position", textprops={"ha": "center"}, width=0.15, group="Player")
team_column = ColumnDefinition("Team", plot_fn=image, width=0.15, textprops={"ha": "center"}, group="Player")
disguise_column = ColumnDefinition("Disguise Score",
            cmap=normed_cmap(df["Disguise Score"], cmap=cmap_red_to_white), width=0.4)

fig, ax = plt.subplots()
plt.tight_layout(pad=0.5)
table = Table(df, column_definitions=[name_column, position_column, team_column, disguise_column])
fig.savefig("../reports/figures/table_1_right.png", facecolor=ax.get_facecolor(), dpi=200, bbox_inches='tight')
plt.close(fig)